# Script for Evaluating Model Runs

This script processes the results from Neptune.ai and evaluates the performance and efficiency of the extensions.

In [1]:
import glob
import os
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

In [2]:
in_dir = Path('/mnt/d/UniHeidelberg/Kurse/Masterarbeit/results')

extension = {
    'ext4': (1, 3),
    'ext3': (4, 6),
    'ext2': (7, 9),
    'ext1': (10, 12),
    'baseline': (13, 15)
}

cmap_diverging = 'BrBG'
colors = plt.get_cmap(cmap_diverging)(np.linspace(0.5, 1.0, plt.get_cmap(cmap_diverging).N // 2))
cmap_sequential = LinearSegmentedColormap.from_list('my_sequential', colors)

## Hardware utilization

In [4]:
column_names = ['measurements', 'time', 'value'] 

hardware_list = ['gpu', 'cpu', 'ram', 'vram']

In [7]:
for hardware in hardware_list:
    data_dir = in_dir / 'performance' / 'hardware' / hardware
    results = []
    for i in range(1,16):
        df = pd.read_csv(data_dir / f'netint_{hardware}{i}.csv', header=None, names=column_names)
        average = df['value'].mean()
        max = df['value'].max()

        results.append({
            'name': i,
            'average': round(average, 2),
            'max': round(max, 2),
        })

    results_df = pd.DataFrame(results)
    results_df.to_csv(in_dir / 'performance' / f'{hardware}.csv', index=False)

In [9]:
for hardware in hardware_list:
    data_dir = in_dir / 'performance' / 'hardware' / hardware
    grouped_results = []
    for group_name, (start, end) in extension.items():
        total_average = 0
        total_max = 0
        count = 0

        for i in range(start, end + 1):
            df = pd.read_csv(data_dir / f'netint_{hardware}{i}.csv', header=None, names=column_names)
            average = df['value'].mean()
            max = df['value'].max()

            total_average += average
            total_max += max
            count += 1

        mean_average = total_average / count
        mean_max = total_max / count

        grouped_results.append({
            'group': group_name,
            'average': round(mean_average, 2),
            'averaged_max': round(mean_max, 2),
        })

    results_avg_df = pd.DataFrame(grouped_results)
    results_avg_df.to_csv(in_dir / 'performance' / 'hardware' / f'{hardware}_grouped.csv', index=False)

### Postprocessing

In [77]:
data_dir = in_dir / 'performance'

In [78]:
df_cpu = pd.read_csv(data_dir / 'cpu.csv')
df_gpu = pd.read_csv(data_dir / 'gpu.csv')
df_ram = pd.read_csv(data_dir / 'ram.csv')
df_vram = pd.read_csv(data_dir / 'vram.csv')

dfs = [df_cpu, df_gpu, df_ram, df_vram]
filenames = ['cpu.csv', 'gpu.csv', 'ram.csv', 'vram.csv']
filenames_no_ext = [os.path.splitext(name)[0] for name in filenames]

for df, name in zip(dfs, filenames_no_ext):
    df.columns = [f"{name}_{col}" if col != 'name' else col for col in df.columns]

df_merged = reduce(lambda left, right: pd.merge(left, right, on='name', how='outer'), dfs)
print(df_merged.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{rrrrrrrrr}
\toprule
name & cpu_average & cpu_max & gpu_average & gpu_max & ram_average & ram_max & vram_average & vram_max \\
\midrule
1 & 24.31 & 70.20 & 8.13 & 63.00 & 17.77 & 27.94 & 3.32 & 3.91 \\
2 & 25.08 & 71.40 & 6.54 & 65.00 & 18.07 & 28.72 & 3.47 & 3.90 \\
3 & 24.57 & 73.90 & 7.30 & 61.00 & 16.98 & 28.25 & 3.26 & 3.90 \\
4 & 23.38 & 61.70 & 10.21 & 52.00 & 11.61 & 19.99 & 2.54 & 3.50 \\
5 & 24.72 & 62.70 & 9.84 & 66.00 & 12.52 & 20.23 & 2.66 & 3.50 \\
6 & 24.11 & 67.20 & 12.65 & 75.00 & 12.83 & 21.40 & 2.65 & 3.49 \\
7 & 26.39 & 66.70 & 9.30 & 69.00 & 14.78 & 25.49 & 3.02 & 3.65 \\
8 & 24.33 & 66.80 & 10.42 & 65.00 & 13.95 & 24.69 & 2.86 & 3.65 \\
9 & 26.42 & 59.90 & 10.51 & 65.00 & 15.27 & 22.60 & 3.14 & 3.67 \\
10 & 30.71 & 70.20 & 14.87 & 63.00 & 11.96 & 18.79 & 2.44 & 3.28 \\
11 & 24.96 & 74.60 & 11.53 & 58.00 & 9.87 & 18.60 & 2.12 & 3.28 \\
12 & 33.91 & 69.60 & 16.72 & 68.00 & 13.57 & 18.51 & 2.69 & 3.28 \\
13 & 34.39 & 79.40 & 18.92 & 67.00 & 8.79 & 13.7

In [79]:
df_merged.set_index('name', inplace=True)

In [80]:
grouped_results = []

for group_name, (start, end) in extension.items():
    selected_rows = df_merged.loc[[start, start + 1, end]]
    for hardware in hardware_list:
        selected_rows[f'{hardware}_average'] = round(selected_rows[f'{hardware}_average'].mean(), 2)
        selected_rows[f'{hardware}_averaged_max'] = round(selected_rows[f'{hardware}_max'].mean(), 2)
    
    grouped_results.append({
        'group': group_name,
        'cpu_average': selected_rows['cpu_average'].mean(),
        'cpu_averaged_max': selected_rows['cpu_averaged_max'].mean(),
        'ram_average': selected_rows['ram_average'].mean(),
        'ram_averaged_max': selected_rows['ram_averaged_max'].mean(),
        'gpu_average': selected_rows['gpu_average'].mean(),
        'gpu_averaged_max': selected_rows['gpu_averaged_max'].mean(),
        'vram_average': selected_rows['vram_average'].mean(),
        'vram_averaged_max': selected_rows['vram_averaged_max'].mean(),
    })

results_avg_df = pd.DataFrame(grouped_results)
results_avg_df.to_csv(in_dir / 'performance' / 'grouped.csv', index=False)
print(results_avg_df.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{lrrrrrrrr}
\toprule
group & cpu_average & cpu_averaged_max & ram_average & ram_averaged_max & gpu_average & gpu_averaged_max & vram_average & vram_averaged_max \\
\midrule
ext4 & 24.65 & 71.83 & 17.61 & 28.30 & 7.32 & 63.00 & 3.35 & 3.90 \\
ext3 & 24.07 & 63.87 & 12.32 & 20.54 & 10.90 & 64.33 & 2.62 & 3.50 \\
ext2 & 25.71 & 64.47 & 14.67 & 24.26 & 10.08 & 66.33 & 3.01 & 3.66 \\
ext1 & 29.86 & 71.47 & 11.80 & 18.63 & 14.37 & 63.00 & 2.42 & 3.28 \\
baseline & 32.30 & 78.60 & 8.49 & 13.97 & 19.40 & 64.33 & 2.31 & 3.28 \\
\bottomrule
\end{tabular}



## Overview

In [10]:
df = pd.read_excel(in_dir / 'overview.xlsx')
df

,name,type,num_channels,epochs,time,size,loss,acc,iou
0,NETINT-1,Extension 4,24,23,1583,27.00,0.659684,0.680103,0.381412
1,NETINT-2,Extension 4,24,33,2231,38.41,0.738906,0.661950,0.370148
2,NETINT-3,Extension 4,24,20,1405,24.54,0.690955,0.746348,0.386134
3,NETINT-4,Extension 3,13,12,535,16.43,1.110041,0.643125,0.282731
4,NETINT-5,Extension 3,13,16,680,21.73,0.968892,0.693708,0.338009
5,NETINT-6,Extension 3,13,16,703,21.26,0.789319,0.697343,0.340236
6,NETINT-7,Extension 2,16,22,1099,29.62,0.714669,0.726199,0.374376
7,NETINT-8,Extension 2,16,16,826,20.98,0.701175,0.705611,0.392931
8,NETINT-9,Extension 2,16,29,1444,40.47,0.735504,0.688995,0.374205
9,NETINT-10,Extension 1,6,25,609,68.02,0.824256,0.688586,0.386819


In [11]:
df['acc'] = df['acc'].apply(lambda x: round(x * 100, 2))
df['iou'] = df['iou'].apply(lambda x: round(x * 100, 2))

In [13]:
print(df.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{llrrrrrrr}
\toprule
name & type & num_channels & epochs & time & size & loss & acc & iou \\
\midrule
NETINT-1 & Extension 4 & 24 & 23 & 1583 & 27.00 & 0.66 & 68.01 & 38.14 \\
NETINT-2 & Extension 4 & 24 & 33 & 2231 & 38.41 & 0.74 & 66.20 & 37.01 \\
NETINT-3 & Extension 4 & 24 & 20 & 1405 & 24.54 & 0.69 & 74.63 & 38.61 \\
NETINT-4 & Extension 3 & 13 & 12 & 535 & 16.43 & 1.11 & 64.31 & 28.27 \\
NETINT-5 & Extension 3 & 13 & 16 & 680 & 21.73 & 0.97 & 69.37 & 33.80 \\
NETINT-6 & Extension 3 & 13 & 16 & 703 & 21.26 & 0.79 & 69.73 & 34.02 \\
NETINT-7 & Extension 2 & 16 & 22 & 1099 & 29.62 & 0.71 & 72.62 & 37.44 \\
NETINT-8 & Extension 2 & 16 & 16 & 826 & 20.98 & 0.70 & 70.56 & 39.29 \\
NETINT-9 & Extension 2 & 16 & 29 & 1444 & 40.47 & 0.74 & 68.90 & 37.42 \\
NETINT-10 & Extension 1 & 6 & 25 & 609 & 68.02 & 0.82 & 68.86 & 38.68 \\
NETINT-11 & Extension 1 & 6 & 13 & 348 & 35.55 & 0.83 & 70.74 & 35.34 \\
NETINT-12 & Extension 1 & 6 & 43 & 1030 & 115.58 & 0.84 & 72.57 & 37.24 \\


In [83]:
grouped_results = []

for group_name, (start, end) in extension.items():
    total_epochs = 0
    total_time = 0
    total_size = 0
    total_loss = 0
    total_acc = 0
    total_iou = 0
    count = 0

    for i in range(start, end + 1):
        df_ex = df.loc[df['name'] == f'NETINT-{i}']

        epochs = int(df_ex['epochs'].iloc[0])
        time = int(df_ex['time'].iloc[0])
        size = float(df_ex['size'].iloc[0])
        loss = float(df_ex['loss'].iloc[0])
        acc = float(df_ex['acc'].iloc[0])
        iou = float(df_ex['iou'].iloc[0])

        total_epochs += epochs
        total_time += time
        total_size += size
        total_loss += loss
        total_acc += acc
        total_iou += iou
        count += 1

    mean_epochs = total_epochs / count
    mean_time = total_time / count
    mean_size = total_size / count
    mean_loss = (total_loss / count)
    mean_acc = (total_acc / count)
    mean_iou = (total_iou / count)

    grouped_results.append({
        'group': group_name,
        'mean_epochs': round(mean_epochs, 2),
        'mean_time': round(mean_time, 2),
        'mean_size': round(mean_size, 2),
        'mean_loss': round(mean_loss * 100, 2),
        'mean_acc': round(mean_acc * 100, 2),
        'mean_iou': round(mean_iou * 100, 2)
    })

results_df = pd.DataFrame(grouped_results)
results_df.sort_values('group', ascending=True, inplace=True)
print(results_df.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{lrrrrrr}
\toprule
group & mean_epochs & mean_time & mean_size & mean_loss & mean_acc & mean_iou \\
\midrule
baseline & 26.00 & 498.67 & 59.69 & 91.43 & 71.63 & 33.45 \\
ext1 & 27.00 & 662.33 & 73.05 & 83.22 & 70.72 & 37.09 \\
ext2 & 22.33 & 1123.00 & 30.36 & 71.71 & 70.69 & 38.05 \\
ext3 & 14.67 & 639.33 & 19.81 & 95.61 & 67.81 & 32.03 \\
ext4 & 25.33 & 1739.67 & 29.98 & 69.65 & 69.61 & 37.92 \\
\bottomrule
\end{tabular}



## Performance Overview

In [3]:
df = pd.read_csv(in_dir / 'performance' / 'performance_overview.csv')
df

,Id,Tags,num_channels,train/epoch/acc (max),val/epoch/acc (max),test/epoch/acc (max),train/epoch/iou (max),val/epoch/iou (max),test/epoch/iou (max)
0,NETINT-1,"[""Extension 4""]",24,0.681084,0.761104,0.680103,0.377427,0.378043,0.381412
1,NETINT-2,"[""Extension 4""]",24,0.693923,0.778450,0.661950,0.398840,0.447149,0.370148
2,NETINT-3,"[""Extension 4""]",24,0.675723,0.726466,0.746348,0.369385,0.387999,0.386134
3,NETINT-4,"[""Extension 3""]",13,0.651914,0.683902,0.643125,0.346230,0.348350,0.282731
4,NETINT-5,"[""Extension 3""]",13,0.666901,0.682022,0.693708,0.355972,0.355555,0.338009
5,NETINT-6,"[""Extension 3""]",13,0.656860,0.727253,0.697343,0.342502,0.392528,0.340236
6,NETINT-7,"[""Extension 2""]",16,0.675677,0.697451,0.726199,0.373616,0.380536,0.374376
7,NETINT-8,"[""Extension 2""]",16,0.662083,0.775494,0.705611,0.358631,0.388841,0.392931
8,NETINT-9,"[""Extension 2""]",16,0.712303,0.737742,0.688995,0.396588,0.425720,0.374205
9,NETINT-10,"[""Extension 1""]",6,0.661805,0.718829,0.688586,0.359877,0.335924,0.386819


In [4]:
for col in df.columns[3:]:
    df[col] = df[col].apply(lambda x: round(x * 100, 2))

In [5]:
print(df.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{llrrrrrrr}
\toprule
Id & Tags & num_channels & train/epoch/acc (max) & val/epoch/acc (max) & test/epoch/acc (max) & train/epoch/iou (max) & val/epoch/iou (max) & test/epoch/iou (max) \\
\midrule
NETINT-1 & ["Extension 4"] & 24 & 68.11 & 76.11 & 68.01 & 37.74 & 37.80 & 38.14 \\
NETINT-2 & ["Extension 4"] & 24 & 69.39 & 77.85 & 66.19 & 39.88 & 44.71 & 37.01 \\
NETINT-3 & ["Extension 4"] & 24 & 67.57 & 72.65 & 74.63 & 36.94 & 38.80 & 38.61 \\
NETINT-4 & ["Extension 3"] & 13 & 65.19 & 68.39 & 64.31 & 34.62 & 34.83 & 28.27 \\
NETINT-5 & ["Extension 3"] & 13 & 66.69 & 68.20 & 69.37 & 35.60 & 35.56 & 33.80 \\
NETINT-6 & ["Extension 3"] & 13 & 65.69 & 72.73 & 69.73 & 34.25 & 39.25 & 34.02 \\
NETINT-7 & ["Extension 2"] & 16 & 67.57 & 69.75 & 72.62 & 37.36 & 38.05 & 37.44 \\
NETINT-8 & ["Extension 2"] & 16 & 66.21 & 77.55 & 70.56 & 35.86 & 38.88 & 39.29 \\
NETINT-9 & ["Extension 2"] & 16 & 71.23 & 73.77 & 68.90 & 39.66 & 42.57 & 37.42 \\
NETINT-10 & ["Extension 1"] & 6 & 66.18 & 

In [11]:
grouped_results = []

for group_name, (start, end) in extension.items():
    num_channels = 0
    total_train_acc = 0
    total_val_acc = 0
    total_test_acc = 0
    total_train_iou = 0
    total_val_iou = 0
    total_test_iou = 0
    count = 0

    for i in range(start, end + 1):
        df_ex = df.loc[df['Id'] == f'NETINT-{i}']

        num_channels = int(df_ex['num_channels'].iloc[0])
        train_acc = float(df_ex['train/epoch/acc (max)'].iloc[0])
        val_acc = float(df_ex['val/epoch/acc (max)'].iloc[0])
        test_acc = float(df_ex['test/epoch/acc (max)'].iloc[0])
        train_iou = float(df_ex['train/epoch/iou (max)'].iloc[0])
        val_iou = float(df_ex['val/epoch/iou (max)'].iloc[0])
        test_iou = float(df_ex['test/epoch/iou (max)'].iloc[0])

        total_train_acc += train_acc
        total_val_acc += val_acc    
        total_test_acc += test_acc
        total_train_iou += train_iou
        total_val_iou += val_iou
        total_test_iou += test_iou
        count += 1

    grouped_results.append({
        'group': group_name,
        'num_channels': num_channels,
        'mean_train_acc': round(total_train_acc / count, 2),
        'mean_val_acc': round(total_val_acc / count, 2),
        'mean_test_acc': round(total_test_acc / count, 2),
        'mean_train_iou': round(total_train_iou / count, 2),
        'mean_val_iou': round(total_val_iou / count, 2),
        'mean_test_iou': round(total_test_iou / count, 2)
    })

results_df = pd.DataFrame(grouped_results)
results_df.sort_values('group', ascending=True, inplace=True)
print(results_df.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{lrrrrrrr}
\toprule
group & num_channels & mean_train_acc & mean_val_acc & mean_test_acc & mean_train_iou & mean_val_iou & mean_test_iou \\
\midrule
baseline & 5 & 66.46 & 68.53 & 71.63 & 35.09 & 35.37 & 33.45 \\
ext1 & 6 & 66.92 & 69.94 & 70.72 & 36.40 & 34.55 & 37.09 \\
ext2 & 16 & 68.34 & 73.69 & 70.69 & 37.63 & 39.83 & 38.05 \\
ext3 & 13 & 65.86 & 69.77 & 67.80 & 34.82 & 36.55 & 32.03 \\
ext4 & 24 & 68.36 & 75.54 & 69.61 & 38.19 & 40.44 & 37.92 \\
\bottomrule
\end{tabular}



In [12]:
results_df.to_csv(in_dir / 'performance' / 'performance_avg.csv', index=False)

## OA and IoU with Hardcut at Baseline Epochs

In [18]:
data_dir = in_dir / 'performance'

column_names = ['step', 'time', 'value']
epoch_cut = 12

In [24]:
results = []

for i in range(1,16):
    train_values = []
    val_values = []
    for metric in ['acc', 'iou']:
        df_train = pd.read_csv(data_dir / metric / 'train' / f'NETINT-{i}__train_epoch_{metric}.csv', header=None, names=column_names)
        df_val = pd.read_csv(data_dir / metric / 'val' / f'NETINT-{i}__val_epoch_{metric}.csv', header=None, names=column_names)

        merge = df_train.merge(df_val, on='step', suffixes=('_train', '_val'), how='outer')
        merge.drop(columns=['time_train', 'time_val'], inplace=True)

        merge['value_train'] = merge['value_train'] * 100
        merge['value_val'] = merge['value_val'] * 100
        
        if epoch_cut in merge['step'].values:
            train_cut = merge.loc[merge['step'] == epoch_cut, 'value_train'].iloc[0]
            val_cut = merge.loc[merge['step'] == epoch_cut, 'value_val'].iloc[0]
            step = epoch_cut
        else:
            step = merge['step'].max()
            train_cut = merge.loc[merge['step'] == step, 'value_train'].iloc[0]
            val_cut = merge.loc[merge['step'] == step, 'value_val'].iloc[0]

        train_values.append(train_cut)
        val_values.append(val_cut)

    results.append({
        'name': i,
        'step': step,
        'oa_train_cut': round(train_values[0], 2),
        'oa_val_cut': round(val_values[0], 2),
        'iou_train_cut': round(train_values[1], 2),
        'iou_val_cut': round(val_values[1], 2),
    })

results_df = pd.DataFrame(results)
results_df.sort_values('name', ascending=True, inplace=True)
print(results_df.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{rrrrrr}
\toprule
name & step & oa_train_cut & oa_val_cut & iou_train_cut & iou_val_cut \\
\midrule
1 & 12.00 & 64.61 & 62.35 & 35.14 & 30.74 \\
2 & 12.00 & 66.35 & 66.19 & 36.28 & 37.53 \\
3 & 12.00 & 65.94 & 67.49 & 35.50 & 35.08 \\
4 & 11.00 & 64.62 & 62.50 & 33.08 & 31.16 \\
5 & 12.00 & 64.65 & 62.33 & 34.18 & 32.49 \\
6 & 12.00 & 62.75 & 64.06 & 33.74 & 36.72 \\
7 & 12.00 & 65.97 & 63.75 & 35.41 & 33.71 \\
8 & 12.00 & 66.21 & 68.71 & 35.86 & 34.90 \\
9 & 12.00 & 65.32 & 65.76 & 35.51 & 37.36 \\
10 & 12.00 & 61.98 & 69.07 & 31.77 & 30.93 \\
11 & 12.00 & 62.55 & 61.07 & 31.79 & 30.75 \\
12 & 12.00 & 66.82 & 64.18 & 35.60 & 31.11 \\
13 & 12.00 & 61.66 & 58.34 & 31.98 & 31.23 \\
14 & 12.00 & 63.31 & 60.84 & 32.73 & 26.71 \\
15 & 12.00 & 60.97 & 52.55 & 30.41 & 23.83 \\
\bottomrule
\end{tabular}



## Confusion Matrices / Class-Wise Accuracy

In [7]:
data_dir = in_dir / 'conf_matrices'
out_dir = Path('/mnt/d/UniHeidelberg/Kurse/Masterarbeit/writing/Figures/results/extensions/conf_matrices')

In [14]:
results = []

xlsx_files = glob.glob(os.path.join(data_dir, '*.xlsx'))

for file_path in xlsx_files:
    filename = os.path.basename(file_path)
    ext = filename.split('_')[0]
    i = filename.split('_')[1]
    df = pd.read_excel(data_dir / filename, index_col=0)

    plt.figure(figsize=(8, 8))

    ax = sns.heatmap(df, annot=True, cmap='BrBG', square=True, cbar=False)

    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=25, ha='right')

    plt.tight_layout()
    plt.savefig(out_dir / f'{ext}_{i}.png')
    plt.close()

    results.append({
        'group': ext,
        'name': i,
        'built-up': df.loc['built-up', 'built-up'] * 100,
        'forest': df.loc['forest', 'forest'] * 100,
        'water': df.loc['water', 'water'] * 100,
        'farmland': df.loc['farmland', 'farmland'] * 100,
        'permanent_crops': df.loc['permanent_crops', 'permanent_crops'] * 100,
        'grass': df.loc['grass', 'grass'] * 100,
    })

results_df = pd.DataFrame(results)
results_df.sort_values('group', ascending=True, inplace=True)

In [26]:
print(results_df.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{llrrrrrr}
\toprule
group & name & built-up & forest & water & farmland & permanent_crops & grass \\
\midrule
baseline & 13 & 70.00 & 78.00 & 91.00 & 67.00 & 78.00 & 22.00 \\
baseline & 14 & 68.00 & 85.00 & 91.00 & 76.00 & 31.00 & 40.00 \\
baseline & 15 & 73.00 & 90.00 & 52.00 & 64.00 & 20.00 & 62.00 \\
ext1 & 10 & 85.00 & 86.00 & 85.00 & 66.00 & 54.00 & 39.00 \\
ext1 & 11 & 78.00 & 82.00 & 87.00 & 78.00 & 39.00 & 41.00 \\
ext1 & 12 & 83.00 & 86.00 & 87.00 & 78.00 & 60.00 & 24.00 \\
ext2 & 7 & 81.00 & 84.00 & 90.00 & 74.00 & 57.00 & 40.00 \\
ext2 & 8 & 87.00 & 84.00 & 93.00 & 68.00 & 66.00 & 27.00 \\
ext2 & 9 & 90.00 & 84.00 & 92.00 & 63.00 & 47.00 & 51.00 \\
ext3 & 4 & 68.00 & 83.00 & 72.00 & 59.00 & 46.00 & 19.00 \\
ext3 & 5 & 81.00 & 90.00 & 89.00 & 57.00 & 44.00 & 26.00 \\
ext3 & 6 & 88.00 & 88.00 & 91.00 & 55.00 & 70.00 & 16.00 \\
ext4 & 1 & 89.00 & 77.00 & 93.00 & 66.00 & 68.00 & 38.00 \\
ext4 & 2 & 87.00 & 84.00 & 88.00 & 48.00 & 87.00 & 35.00 \\
ext4 & 3 & 87.00 

### Grouped

In [13]:
results = []

for group_name, (start, end) in extension.items():

    df1 = pd.read_excel(data_dir / f"{group_name}_{start}_conf_matrix.xlsx", index_col=0)
    df1 = df1.map(lambda x: pd.to_numeric(x, errors='coerce'))
    df2 = pd.read_excel(data_dir / f"{group_name}_{start+1}_conf_matrix.xlsx", index_col=0)
    df2 = df2.map(lambda x: pd.to_numeric(x, errors='coerce'))
    df3 = pd.read_excel(data_dir / f"{group_name}_{start+2}_conf_matrix.xlsx", index_col=0)
    df3 = df3.map(lambda x: pd.to_numeric(x, errors='coerce'))

    df = round(((df1 + df2 + df3) / 3) * 100, 2)

    results.append({
        'group': group_name,
        'built-up': df.loc['built-up', 'built-up'],
        'forest': df.loc['forest', 'forest'],
        'water': df.loc['water', 'water'],
        'farmland': df.loc['farmland', 'farmland'],
        'permanent_crops': df.loc['permanent_crops', 'permanent_crops'],
        'grass': df.loc['grass', 'grass'],
    })

    plt.figure(figsize=(8, 8))
    ax = sns.heatmap(df, annot=True, cmap='BrBG', square=True, cbar=False)

    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=25, ha='right')

    plt.tight_layout()
    plt.savefig(out_dir / f'{group_name}_mean.png')
    plt.close()

results_df = pd.DataFrame(results)

In [86]:
print(results_df.to_latex(index=False, float_format='%.2f', bold_rows=True))

\begin{tabular}{lrrrrrr}
\toprule
group & built-up & forest & water & farmland & permanent_crops & grass \\
\midrule
ext4 & 87.67 & 82.67 & 90.67 & 64.33 & 67.00 & 37.33 \\
ext3 & 79.00 & 87.00 & 84.00 & 57.00 & 53.33 & 20.33 \\
ext2 & 86.00 & 84.00 & 91.67 & 68.33 & 56.67 & 39.33 \\
ext1 & 82.00 & 84.67 & 86.33 & 74.00 & 51.00 & 34.67 \\
baseline & 70.33 & 84.33 & 78.00 & 69.00 & 43.00 & 41.33 \\
\bottomrule
\end{tabular}



## GPU Power Consumption (disregarded)

In [87]:
data_dir = in_dir / 'energy'

### Results per Run

In [88]:
results = []

for i in range(1,16):
    df = pd.read_csv(data_dir / f'netint{i}.csv')
    total_e_consumption = 0
    total_duration = 0
    p_total = 0
    for j in range(0,4):
        e_consumption = df.iloc[j]['nvidia_gpu_0'] * 0.001
        duration = df.iloc[j]['duration']

        total_e_consumption += e_consumption
        total_duration += duration
        p_total += e_consumption / duration

    results.append({
        'netint': i,
        'e_consumption': total_e_consumption,
        'duration': total_duration,
        'p_total': p_total
    })

results_df = pd.DataFrame(results)
results_df.sort_values('p_total', ascending=False, inplace=True)
results_df

,netint,e_consumption,duration,p_total
14,15,41767.902,689.408828,172.554283
13,14,15339.600,267.317706,167.642594
12,13,31895.890,534.852387,166.991763
11,12,55881.659,1027.094838,160.529276
5,6,29764.425,700.515298,151.076532
9,10,32404.456,606.216156,148.045857
7,8,32946.362,823.690160,145.298611
4,5,29252.158,676.997648,144.810151
10,11,17980.049,345.073310,144.757630
8,9,58354.046,1442.045189,144.399860


In [89]:
results = []

for i in range(1,16):
    df = pd.read_csv(data_dir / f'netint{i}.csv')
    e_consumption = df['nvidia_gpu_0'].sum() * 0.001
    duration = df['duration'].sum()

    p_total = e_consumption / duration

    results.append({
        'netint': i,
        'e_consumption': e_consumption,
        'duration': duration,
        'p_total': p_total
    })

results_df = pd.DataFrame(results)
results_df.sort_values('p_total', ascending=False, inplace=True)
results_df

,netint,e_consumption,duration,p_total
14,15,41767.902,689.408828,60.585099
12,13,31895.890,534.852387,59.634940
13,14,15339.600,267.317706,57.383404
11,12,55881.659,1027.094838,54.407497
9,10,32404.456,606.216156,53.453633
10,11,17980.049,345.073310,52.105012
4,5,29252.158,676.997648,43.208655
3,4,22781.398,532.526691,42.779824
5,6,29764.425,700.515298,42.489329
6,7,44515.070,1097.252371,40.569582


### Grouped results

In [90]:
grouped_results = []

for group_name, (start, end) in extension.items():
    total_e_consumption = 0
    total_duration = 0
    count = 0

    for i in range(start, end + 1):
        df = pd.read_csv(data_dir / f'netint{i}.csv')
        e_consumption = df['nvidia_gpu_0'].sum() * 0.001
        duration = df['duration'].sum()

        total_e_consumption += e_consumption
        total_duration += duration
        count += 1

    mean_e_consumption = total_e_consumption / count
    mean_duration = total_duration / count

    efficiency = mean_e_consumption / mean_duration

    grouped_results.append({
        'group': group_name,
        'mean_e_consumption': mean_e_consumption,
        'mean_duration': mean_duration,
        'efficiency': efficiency
    })

results_avg_df = pd.DataFrame(grouped_results)
results_avg_df.sort_values('efficiency', ascending=True, inplace=True)
results_avg_df

,group,mean_e_consumption,mean_duration,efficiency
0,ext4,62053.264667,1737.073799,35.722872
2,ext2,45271.826000,1120.995907,40.385362
1,ext3,27265.993667,636.679879,42.825279
3,ext1,35422.054667,659.461435,53.713611
4,baseline,29667.797333,497.192974,59.670588
